# ML Experiments using different bin sizes & widths based on outputs from supervised discretization using ChiMerge discretizer
## Dataset: iris

by: Malina & Sam , 06.07.2022

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import six
import sys
sys.modules['sklearn.externals.six'] = six
import id3
from id3 import Id3Estimator
from id3 import export_graphviz
import wittgenstein as lw
#import warnings
#warnings.filterwarnings('ignore')

from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

## EDA
from collections import Counter

# Knn-VDM 3
from vdm3 import ValueDifferenceMetric
from sklearn.neighbors import KNeighborsClassifier

# Pre-processing
from sklearn.preprocessing import OrdinalEncoder
# Cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score # 1 metric
from sklearn.model_selection import cross_validate # more than 1 metric
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
#read in data from ChiMerge discretization
iris6 = pd.read_csv('chim_iris_6int.csv')
iris8 = pd.read_csv('chim_iris_8int.csv')
iris10 = pd.read_csv('chim_iris_10int.csv')
iris15 = pd.read_csv('chim_iris_15int.csv')

In [3]:
iris8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB


## count values for interval frequencies

In [4]:
print(Counter(iris6['slength']))
print(Counter(iris6['swidth']))
print(Counter(iris6['plength']))
print(Counter(iris6['swidth']))

Counter({4: 65, 2: 30, 3: 21, 0: 16, 5: 12, 1: 6})
Counter({4: 57, 5: 36, 2: 36, 3: 10, 1: 7, 0: 4})
Counter({0: 50, 5: 34, 1: 29, 2: 16, 4: 12, 3: 9})
Counter({4: 57, 5: 36, 2: 36, 3: 10, 1: 7, 0: 4})


In [5]:
print(Counter(iris8['slength']))
print(Counter(iris8['swidth']))
print(Counter(iris8['plength']))
print(Counter(iris8['swidth']))

Counter({6: 39, 2: 30, 3: 21, 5: 19, 0: 16, 7: 12, 4: 7, 1: 6})
Counter({4: 57, 2: 36, 5: 18, 6: 12, 3: 10, 1: 7, 7: 6, 0: 4})
Counter({0: 50, 7: 34, 1: 29, 4: 9, 3: 8, 2: 8, 6: 8, 5: 4})
Counter({4: 57, 2: 36, 5: 18, 6: 12, 3: 10, 1: 7, 7: 6, 0: 4})


In [6]:
print(Counter(iris10['slength']))
print(Counter(iris10['swidth']))
print(Counter(iris10['plength']))
print(Counter(iris10['swidth']))

Counter({2: 30, 3: 21, 6: 21, 5: 19, 0: 16, 8: 16, 9: 12, 4: 7, 1: 6, 7: 2})
Counter({2: 36, 5: 31, 4: 26, 8: 12, 6: 12, 3: 10, 1: 7, 7: 6, 9: 6, 0: 4})
Counter({0: 50, 8: 33, 1: 29, 3: 8, 2: 8, 7: 8, 5: 5, 4: 4, 6: 4, 9: 1})
Counter({2: 36, 5: 31, 4: 26, 8: 12, 6: 12, 3: 10, 1: 7, 7: 6, 9: 6, 0: 4})


In [7]:
print(Counter(iris15['slength']))
print(Counter(iris15['swidth']))
print(Counter(iris15['plength']))
print(Counter(iris15['swidth']))

Counter({2: 30, 0: 16, 9: 16, 12: 15, 7: 15, 14: 12, 5: 8, 6: 7, 3: 7, 1: 6, 4: 6, 10: 5, 8: 4, 11: 2, 13: 1})
Counter({8: 31, 7: 26, 4: 22, 5: 14, 9: 12, 6: 10, 10: 6, 14: 6, 13: 6, 2: 4, 11: 3, 12: 3, 3: 3, 1: 3, 0: 1})
Counter({0: 50, 1: 29, 8: 25, 3: 8, 2: 8, 7: 8, 5: 5, 4: 4, 6: 4, 9: 3, 13: 2, 12: 1, 10: 1, 14: 1, 11: 1})
Counter({8: 31, 7: 26, 4: 22, 5: 14, 9: 12, 6: 10, 10: 6, 14: 6, 13: 6, 2: 4, 11: 3, 12: 3, 3: 3, 1: 3, 0: 1})


In [8]:
iris8

,slength,swidth,plength,pwidth,label
0,2,5,0,0,0
1,1,4,0,0,0
2,0,4,0,0,0
3,0,4,0,0,0
4,2,6,0,0,0
...,...,...,...,...,...
145,6,4,7,7,2
146,6,2,5,7,2
147,6,4,7,7,2
148,5,5,7,7,2


# 1. Categorical Naive Bayes

## 1.1 6 Intervals from ChiMerge

## 1.1 debug version

In [9]:
# X = iris6.drop('label', axis=1)
# y = iris6['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

In [10]:
# import time
# start = time.time() # For measuring time execution

# cnb = CategoricalNB()
# cnb.fit(X_train, y_train)

# y_pred_cnb = cnb.predict(X_test)
# y_prob_pred_cnb = cnb.predict_proba(X_test)

# end = time.time()
# print(end - start) # Total time execution for this sample


In [11]:
# # how did our model perform?
# count_misclassified = (y_test != y_pred_cnb).sum()
# print("CategoricalNB")
# print("=" * 25)
# print('Misclassified samples: {}'.format(count_misclassified))
# accuracy = accuracy_score(y_test, y_pred_cnb)
# print('Accuracy: {:.2f}'.format(accuracy))
# print("=" * 25)
# print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
# print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
# print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
# print("=" * 25)
# print("Classification report:")
# print(classification_report(y_test, y_pred_cnb))

In [12]:
# from sklearn.metrics import zero_one_loss
# #This library is used to decompose bias and variance in our models
# from mlxtend.evaluate import bias_variance_decomp
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.linear_model import LinearRegression, Lasso
# import warnings
# warnings.filterwarnings('ignore')
# #We will load the Boston house dataset for our example
# from sklearn.datasets import load_boston
# from sklearn import metrics

# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

### 10 fold cross validation

In [13]:
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_classification
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# # prepare the cross-validation procedure
# cv = KFold(n_splits=10, random_state=1, shuffle=True)

# # evaluate model
# scores = cross_val_score(cnb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [14]:
# # use repeated k-fold cross-validation - 3 repeats
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_classification
# from sklearn.model_selection import RepeatedKFold
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression

# # prepare the cross-validation procedure
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # create model

# # evaluate model
# scores = cross_val_score(cnb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

## 1.1 script version

In [4]:
# make test & train split
X = iris6.drop('label', axis=1)
y = iris6['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 2
Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Computation time:
0.019899368286132812


In [5]:
# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.048
Average bias: 0.053
Average variance: 0.012
Sklearn 0-1 loss: 0.053


In [6]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

# evaluate model
scores = cross_val_score(cnb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.960 (0.050)


## 1.2 8 Intervals from ChiMerge

In [7]:
# make test & train split
X = iris8.drop('label', axis=1)
y = iris8['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 2
Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Computation time:
0.021968364715576172


In [12]:
# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

IndexError: index 5 is out of bounds for axis 1 with size 5

In [19]:
print(len(X_train.values))
print(len(y_train.values))

print(len(X_test.values))
print(len(y_test.values))

112
112
38
38


In [20]:
# y_pred_cnb
# y_prob_pred_cnb

#avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
      #  cnb, X_train.values, y_train.values, X_test.values, y_test.values, loss='0-1_loss', random_seed=123)

#print('Average expected loss: %.3f' % avg_expected_loss)
#print('Average bias: %.3f' % avg_bias)
#print('Average variance: %.3f' % avg_var)
#print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

In [21]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

# evaluate model
scores = cross_val_score(cnb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.962 (0.051)


## 1.3 10 Intervals from ChiMerge

In [22]:
iris10.

,slength,swidth,plength,pwidth,label
0,2,7,0,0,0
1,1,4,0,0,0
2,0,5,0,0,0
3,0,5,0,0,0
4,2,8,0,0,0
...,...,...,...,...,...
145,8,4,8,7,2
146,6,2,6,7,2
147,6,4,8,7,2
148,5,6,8,7,2


In [10]:
X = iris10.drop('label', axis=1).head(100)
y = iris10['label'].head(100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 0
Accuracy: 1.00
Recall score :  1.0
Precision score :  1.0
F1 score :  1.0
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        14

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

Computation time:
0.017142534255981445


In [11]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

# evaluate model
scores = cross_val_score(cnb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: nan (nan)


## 1.4 15 Intervals from ChiMerge

In [ ]:
iris15

In [16]:
X = iris15.drop('label', axis=1)
y = iris15['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print(end - start) # Total time execution for this sample

IndexError: index 14 is out of bounds for axis 1 with size 14

In [ ]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

# evaluate model
scores = cross_val_score(cnb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# 2. Decision Tree ID3

## 2.1 6 Intervals from ChiMerge

### Debug version

In [13]:
X = iris6.drop('label', axis=1)
y = iris6['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

In [14]:
import time
start = time.time() # For measuring time execution

estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)

y_pred_id3 = estimator.predict(X_test)
#y_prob_pred_id3 = estimator.predict_proba(X_test)

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Computation time:
0.011519908905029297


In [15]:
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38



In [16]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.054
Average bias: 0.053
Average variance: 0.002
Sklearn 0-1 loss: 0.053


### script version

In [17]:
#make splits
X = iris6.drop('label', axis=1)
y = iris6['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Computation time:
0.021643400192260742


In [18]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.054
Average bias: 0.053
Average variance: 0.002
Sklearn 0-1 loss: 0.053


In [19]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

if __name__ == '__main__':
    # evaluate model
    #BUG: n_jobs=-1 does not work
    #SOLVED: n_jobs = 1 --> does not fix it but no error because of praralell processing issue in windows
    # see https://github.com/scikit-learn/scikit-learn/issues/13228
    #scores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    scores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv, n_jobs=1)
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.949 (0.054)


## 2.2 8 Intervals from ChiMerge

In [20]:
#make splits
X = iris8.drop('label', axis=1)
y = iris8['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Computation time:
0.023897886276245117


In [21]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.055
Average bias: 0.053
Average variance: 0.003
Sklearn 0-1 loss: 0.053


In [22]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

if __name__ == '__main__':
    # evaluate model
    #BUG: n_jobs=-1 does not work
    #SOLVED: n_jobs = 1 --> does not fix it but no error because of praralell processing issue in windows
    # see https://github.com/scikit-learn/scikit-learn/issues/13228
    #scores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    cores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv)
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.949 (0.054)


## 2.3 10 Intervals from ChiMerge

In [23]:
#make splits
X = iris10.drop('label', axis=1)
y = iris10['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Computation time:
0.025766849517822266


In [24]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.056
Average bias: 0.053
Average variance: 0.005
Sklearn 0-1 loss: 0.053


In [25]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

if __name__ == '__main__':
    # evaluate model
    #BUG: n_jobs=-1 does not work
    #SOLVED: n_jobs = 1 --> does not fix it but no error because of praralell processing issue in windows
    # see https://github.com/scikit-learn/scikit-learn/issues/13228
    #scores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    cores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv)
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.949 (0.054)


## 2.4 15 Intervals from CHiMerge

In [26]:
#make splits
X = iris15.drop('label', axis=1)
y = iris15['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.95
Recall score :  0.9473684210526315
Precision score :  0.9473684210526315
F1 score :  0.9473684210526315
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Computation time:
0.022498607635498047


In [27]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.064
Average bias: 0.053
Average variance: 0.012
Sklearn 0-1 loss: 0.053


In [28]:
# use repeated k-fold cross-validation - 3 repeats
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model

if __name__ == '__main__':
    # evaluate model
    #BUG: n_jobs=-1 does not work
    #SOLVED: n_jobs = 1 --> does not fix it but no error because of praralell processing issue in windows
    # see https://github.com/scikit-learn/scikit-learn/issues/13228
    #scores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    cores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv)
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.949 (0.054)


# 3. KNN classification

In [29]:
#read in data from ChiMerge discretization
iris6 = pd.read_csv('chim_iris_6int.csv')
iris8 = pd.read_csv('chim_iris_8int.csv')
iris10 = pd.read_csv('chim_iris_10int.csv')
iris15 = pd.read_csv('chim_iris_15int.csv')

## 3.1 KNN with ChiMerge 6 intervals

In [30]:
# Complete code for data preperation
# Read data
df_ewd1 = iris6
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('label', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
(150, 4) (150,)
Class representation - original:  Counter({0: 50, 1: 50, 2: 50})
Class representation - training data:  Counter({1: 39, 0: 38, 2: 35})
Class representation - testing data:  Counter({2: 15, 0: 12, 1: 11})


In [31]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Time for training model Knn-VDM: 1.1522431373596191.


In [32]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

Average expected loss: 0.051
Average bias: 0.053
Average variance: 0.010
Sklearn 0-1 loss: 0.053


## 3.2 KNN with ChiMerge 8 intervals

In [33]:
# Complete code for data preperation
# Read data
df_ewd1 = iris8
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('label', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
(150, 4) (150,)
Class representation - original:  Counter({0: 50, 1: 50, 2: 50})
Class representation - training data:  Counter({1: 39, 0: 38, 2: 35})
Class representation - testing data:  Counter({2: 15, 0: 12, 1: 11})


In [34]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.91      0.91      0.91        11
           2       0.93      0.93      0.93        15

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Time for training model Knn-VDM: 1.4965150356292725.


In [35]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

Average expected loss: 0.062
Average bias: 0.053
Average variance: 0.041
Sklearn 0-1 loss: 0.053


## 3.3 KNN with ChiMerge 10 intervals

In [50]:
# Complete code for data preperation
# Read data
df_ewd1 = iris10.head(149)
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('label', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  149 non-null    int64
 1   swidth   149 non-null    int64
 2   plength  149 non-null    int64
 3   pwidth   149 non-null    int64
 4   label    149 non-null    int64
dtypes: int64(5)
memory usage: 5.9 KB
(149, 4) (149,)
Class representation - original:  Counter({0: 50, 1: 50, 2: 49})
Class representation - training data:  Counter({1: 40, 0: 38, 2: 33})
Class representation - testing data:  Counter({2: 16, 0: 12, 1: 10})


In [51]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.82      0.90      0.86        10
           2       0.93      0.88      0.90        16

    accuracy                           0.92        38
   macro avg       0.92      0.92      0.92        38
weighted avg       0.92      0.92      0.92        38

Time for training model Knn-VDM: 0.757422685623169.


In [52]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

IndexError: index 111 is out of bounds for axis 0 with size 111

## 3.4 KNN with ChiMerge 15 intervals

In [53]:
# Complete code for data preperation
# Read data
df_ewd1 = iris15.head(149)
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('label', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  149 non-null    int64
 1   swidth   149 non-null    int64
 2   plength  149 non-null    int64
 3   pwidth   149 non-null    int64
 4   label    149 non-null    int64
dtypes: int64(5)
memory usage: 5.9 KB
(149, 4) (149,)
Class representation - original:  Counter({0: 50, 1: 50, 2: 49})
Class representation - training data:  Counter({1: 40, 0: 38, 2: 33})
Class representation - testing data:  Counter({2: 16, 0: 12, 1: 10})


In [54]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.77      1.00      0.87        10
           2       1.00      0.81      0.90        16

    accuracy                           0.92        38
   macro avg       0.92      0.94      0.92        38
weighted avg       0.94      0.92      0.92        38

Time for training model Knn-VDM: 0.7466623783111572.


In [55]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train, y_train, X_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

IndexError: index 111 is out of bounds for axis 0 with size 111

# 3. Ripper

## 3.1 Ripper with ChiMerge 6 Intervals

### Script version

In [ ]:
# X = australia8.drop('label', axis=1)
# y = australia8['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)


# #time recording
# import time
# start = time.time() # For measuring time execution

# ripper_clf = lw.RIPPER() # Or irep_clf = lw.IREP() to build a model using IREP
# ripper_clf.fit(X, y) # Or pass X and y data to .fit
# ripper_clf
# ripper_clf.out_model() # or ripper_clf.ruleset_
# ripper_clf.score(X_test, y_test)

# ripper_clf.predict(X_test,  give_reasons=True)
# y_pred_ripper = ripper_clf.predict(X_test)
# ripper_clf.predict_proba(X_test)

# ripper_clf.predict(X_test,  give_reasons=True)
# y_pred_ripper = ripper_clf.predict(X_test)
# ripper_clf.predict_proba(X_test)

# print("Classification report:")
# print(classification_report(y_test, y_pred_ripper))
# #stop time recoridng
# end = time.time()
# print("Computation time:")
# print(end - start) # Total time execution for this sample


In [ ]:
# # use repeated k-fold cross-validation - 3 repeats
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_classification
# from sklearn.model_selection import RepeatedKFold
# from sklearn.model_selection import cross_val_score

# # prepare the cross-validation procedure
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # create model

# # evaluate model
# scores = cross_val_score(ripper_clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))